In [1]:
# Dependencies

import pandas as pd
from pathlib import Path
import requests
from pprint import pprint


Upload csv file

In [2]:
# Store filepath in a variable
file_one = Path("Resources/us_counties_covid19_daily.csv")

In [3]:
# Read our data file with the Pandas library
covid_df = pd.read_csv(file_one, encoding="ISO-8859-1")

In [4]:
# Show the first five rows.
covid_df.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [5]:
# Select rows in my date range
# Make a start date
start_date = '2020-09-01'

# Make an end date
end_date = '2020-11-30'

# Select DataFrame rows between two dates
mask = (covid_df['date'] > start_date) & (covid_df['date'] <= end_date)
fall_covid_df = covid_df.loc[mask]

fall_covid_df

,date,county,state,fips,cases,deaths
492429,2020-09-02,Autauga,Alabama,1001.0,1452,24.0
492430,2020-09-02,Baldwin,Alabama,1003.0,4583,42.0
492431,2020-09-02,Barbour,Alabama,1005.0,768,7.0
492432,2020-09-02,Bibb,Alabama,1007.0,562,8.0
492433,2020-09-02,Blount,Alabama,1009.0,1308,11.0
...,...,...,...,...,...,...
784205,2020-11-30,Sweetwater,Wyoming,56037.0,1888,6.0
784206,2020-11-30,Teton,Wyoming,56039.0,1644,2.0
784207,2020-11-30,Uinta,Wyoming,56041.0,1070,4.0
784208,2020-11-30,Washakie,Wyoming,56043.0,440,8.0


In [6]:
# Add the weather column and city to out DataFrame
fall_covid_df['Temperature (C)'] = ''
#fall_covid_df['City'] = ''
fall_covid_df

C:\Users\Ryan Kracaw\AppData\Local\Temp\ipykernel_26556\1038689862.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fall_covid_df['Temperature (C)'] = ''


,date,county,state,fips,cases,deaths,Temperature (C)
492429,2020-09-02,Autauga,Alabama,1001.0,1452,24.0,
492430,2020-09-02,Baldwin,Alabama,1003.0,4583,42.0,
492431,2020-09-02,Barbour,Alabama,1005.0,768,7.0,
492432,2020-09-02,Bibb,Alabama,1007.0,562,8.0,
492433,2020-09-02,Blount,Alabama,1009.0,1308,11.0,
...,...,...,...,...,...,...,...
784205,2020-11-30,Sweetwater,Wyoming,56037.0,1888,6.0,
784206,2020-11-30,Teton,Wyoming,56039.0,1644,2.0,
784207,2020-11-30,Uinta,Wyoming,56041.0,1070,4.0,
784208,2020-11-30,Washakie,Wyoming,56043.0,440,8.0,


In [ ]:
asldjakldfh;asdhfl

In [7]:
api_key = '834e67a2eb83f6a885c0eb808cc81d6b'
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"